In [2]:
import pandas as pd
import plotly.graph_objects as go

# Read the data
social_spending = pd.read_csv("/Users/kendallgilbert/Desktop/social_spending.csv", header=1)
gdp = pd.read_csv("/Users/kendallgilbert/Desktop/DSAN5200-Project/5200 Project/Data_cleaned/gdp.csv")

# Clean and merge the data
social_spending = social_spending.rename(columns={'Year': 'Country'})
social_spending = social_spending.applymap(lambda x: x.strip() if isinstance(x, str) else x)
social_spending = pd.melt(social_spending, id_vars=['Country'], var_name='Year', value_name='s_s')
social_spending['Country_Year'] = social_spending['Country'] + '_' + social_spending['Year'].astype(str)

merged_data = pd.merge(gdp, social_spending, on=['Country_Year'])
merged_data['gdp'] = pd.to_numeric(merged_data['gdp'], errors='coerce')
merged_data['s_s'] = pd.to_numeric(merged_data['s_s'], errors='coerce')
merged_data['gdp'] = merged_data['gdp'].fillna(merged_data['gdp'].mean())
merged_data['s_s'] = merged_data['s_s'].fillna(merged_data['s_s'].mean())


# Calculate the average social spending over all years for each country
average_social_spending = merged_data.groupby('Country')['s_s'].mean().reset_index()

# Calculate the average GDP over all years for each country
average_gdp = merged_data.groupby('Country')['gdp'].mean().reset_index()

# Merge the average social spending and average GDP data
average_data = pd.merge(average_social_spending, average_gdp, on='Country', suffixes=('_s_s', '_gdp'))

# Calculate the ratio of average social spending to average GDP
average_data['s_s_gdp_ratio'] = average_data['s_s'] / average_data['gdp']

color_map = {
    'Australia': 'rgb(255, 0, 0)',        # Red
    'Austria': 'rgb(0, 255, 0)',          # Green
    'Belgium': 'rgb(0, 0, 255)',          # Blue
    'Canada': 'rgb(255, 255, 0)',         # Yellow
    'Chile': 'rgb(255, 0, 255)',          # Magenta
    'Colombia': 'rgb(0, 255, 255)',       # Cyan
    'Costa Rica': 'rgb(255, 127, 0)',     # Orange
    'Czechia': 'rgb(255, 0, 127)',        # Reddish-Pink
    'Denmark': 'rgb(0, 127, 255)',        # Bluish
    'Estonia': 'rgb(127, 255, 0)',        # Greenish-Yellow
    'Finland': 'rgb(0, 255, 127)',        # Greenish
    'France': 'rgb(127, 0, 255)',         # Purplish-Blue
    'Germany': 'rgb(255, 255, 127)',      # Light Yellow
    'Greece': 'rgb(127, 255, 255)',       # Light Cyan
    'Hungary': 'rgb(255, 127, 255)',      # Light Magenta
    'Iceland': 'rgb(255, 0, 0)',          # Red
    'Ireland': 'rgb(0, 255, 0)',          # Green
    'Israel': 'rgb(0, 0, 255)',           # Blue
    'Italy': 'rgb(255, 255, 0)',          # Yellow
    'Japan': 'rgb(255, 0, 255)',          # Magenta
    'Korea': 'rgb(0, 255, 255)',          # Cyan
    'Latvia': 'rgb(255, 127, 0)',         # Orange
    'Lithuania': 'rgb(255, 0, 127)',      # Reddish-Pink
    'Luxembourg': 'rgb(0, 127, 255)',     # Bluish
    'Mexico': 'rgb(127, 255, 0)',         # Greenish-Yellow
    'Netherlands': 'rgb(0, 255, 127)',    # Greenish
    'New Zealand': 'rgb(127, 0, 255)',    # Purplish-Blue
    'Norway': 'rgb(255, 255, 127)',      # Light Yellow
    'Poland': 'rgb(127, 255, 255)',       # Light Cyan
    'Portugal': 'rgb(255, 127, 255)',     # Light Magenta
    'Slovak Republic': 'rgb(255, 0, 0)',  # Red
    'Slovenia': 'rgb(0, 255, 0)',         # Green
    'Spain': 'rgb(0, 0, 255)',            # Blue
    'Sweden': 'rgb(255, 255, 0)',         # Yellow
    'Switzerland': 'rgb(255, 0, 255)',    # Magenta
    'Türkiye': 'rgb(0, 255, 255)',        # Cyan
    'United Kingdom': 'rgb(255, 127, 0)', # Orange
    'United States': 'rgb(255, 0, 127)',  # Reddish-Pink
    # Add more country-color mappings as needed
}

average_data['color'] = average_data['Country'].map(color_map)

# Define bubble sizes based on the ratio of average social spending to GDP
bubble_sizes = average_data['s_s_gdp_ratio'] * 1000

# Create the bubble chart
fig = go.Figure(data=go.Scatter(
    x=average_data['s_s_gdp_ratio'],  # Swapping X and Y values
    y=average_data['gdp'],  # Swapping X and Y values
    mode='markers',
    marker=dict(
        size=bubble_sizes,
        sizemode='area',
        sizeref=2.*max(bubble_sizes)/(100**2),  # Reference scale for bubble sizes
        sizemin=4,  # Minimum size of the bubble
        color=average_data['color']
    ),
    text=average_data.apply(lambda row: f"Country: {row['Country']}<br>Avg Social Spending: {row['s_s']:,}<br>Avg GDP: {row['gdp']:,}<br>Ratio: {row['s_s_gdp_ratio']:.4f}", axis=1),  # Adjust format to include four decimal places
))

# Update layout
fig.update_layout(
    title='Bubble Chart: Average Social Spending vs. GDP',
    xaxis=dict(title='% of GDP spent towards social benefits'),  # Adjust X-axis label
    yaxis=dict(title='Average GDP'),  # Adjust Y-axis label
    showlegend=False
)

# Show the plot
fig.show()


/var/folders/x3/1m329lgj7sl4y1pqkx_p_f_80000gn/T/ipykernel_54529/3726998272.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  social_spending = social_spending.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [85]:
us_data = average_data[average_data['Country'] == 'United States']
print("United States Results:")
print("Average Social Spending:", us_data['s_s'].values[0])
print("Average GDP:", us_data['gdp'].values[0])
print("Ratio of Average Social Spending to GDP:", us_data['s_s_gdp_ratio'].values[0])


United States Results:
Average Social Spending: 1.3861360718870348
Average GDP: 280.074137254902
Ratio of Average Social Spending to GDP: 0.004949175548563701


In [4]:
iceland_data = average_data[average_data['Country'] == 'Iceland']
print("IcelandResults:")
print("Average Social Spending:", iceland_data['s_s'].values[0])
print("Average GDP:", iceland_data['gdp'].values[0])
print("Ratio of Average Social Spending to GDP:", iceland_data['s_s_gdp_ratio'].values[0])

IcelandResults:
Average Social Spending: 3.0229781771501925
Average GDP: 13.97578947368421
Ratio of Average Social Spending to GDP: 0.2163010671305779
